In [3]:
'''机械臂API测试'''
from Robotic_Arm.rm_robot_interface import *
from ctypes import c_float

# 实例化RoboticArm类
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
# 创建机械臂连接，打印连接id
handle = arm.rm_create_robot_arm("169.254.128.19", 8080)
arm_model = rm_robot_arm_model_e.RM_MODEL_RM_75_E  
force_type = rm_force_type_e.RM_MODEL_RM_B_E  
algo = Algo(arm_model, force_type)

print(arm.rm_get_current_arm_state())
print("-------------------------------------------")
print(arm.rm_get_joint_degree())
print("-------------------------------------------")
print(arm.rm_get_robot_info())


arm.rm_delete_robot_arm()

current c api version:  1.1.3


[rm_create_robot_arm] socket connect err!


KeyboardInterrupt: 

In [ ]:
'''夹抓API测试'''
from Robotic_Arm.rm_robot_interface import *

# 实例化RoboticArm类
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)

# 创建机械臂连接
handle = arm.rm_create_robot_arm("169.254.128.19", 8080)

# 设置夹爪开口的最大值和最小值
#print(arm.rm_set_gripper_route(0, 1000))

# 松开夹爪到开口最大处
#print(arm.rm_set_gripper_release(500, True, 10))

# 夹爪达到指定位置
print(arm.rm_set_gripper_position(1000, True, 10))


arm.rm_delete_robot_arm()

current c api version:  1.1.0
0


0

In [1]:
'''只控制机械臂的姿态
Z轴=腕→指根
X轴=腕→拇指投影
Y轴=叉乘'''
import cv2
import numpy as np
import mediapipe as mp
import pyrealsense2 as rs
from Robotic_Arm.rm_robot_interface import *

# —— 1. 初始化 RealSense —— 
pipeline = rs.pipeline()
cfg = rs.config()
# 分辨率
cfg.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
cfg.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
profile = pipeline.start(cfg)
# 对齐：让 depth 和 color 拥有相同的内参/像素对齐
align = rs.align(rs.stream.color)

# 拿内参
color_stream = profile.get_stream(rs.stream.color)
intr = color_stream.as_video_stream_profile().get_intrinsics()
fx, fy, cx, cy = intr.fx, intr.fy, intr.ppx, intr.ppy

# —— 2. 初始化机械臂 —— 
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
arm_handle = arm.rm_create_robot_arm("192.168.1.18", 8080)
arm_model = rm_robot_arm_model_e.RM_MODEL_RM_65_E
force_type = rm_force_type_e.RM_MODEL_RM_B_E
algo = Algo(arm_model=arm_model, force_type=force_type)

# 固定末端位置
pos0 = [-0.263033, -0.014339, 0.318728]

# —— 3. 手–眼标定旋转矩阵（只用旋转） —— 
q = np.array([-0.464607, 0.552742, -0.525271, 0.450230])
w, x, y, z = q
R_rc = np.array([
    [1-2*(y*y+z*z),   2*(x*y - z*w),   2*(x*z + y*w)],
    [2*(x*y + z*w),   1-2*(x*x+z*z),   2*(y*z - x*w)],
    [2*(x*z - y*w),   2*(y*z + x*w),   1-2*(x*x+y*y)]
])

# —— 4. MediaPipe Hands —— 
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.8, min_tracking_confidence=0.8, max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

def normalize(v: np.ndarray) -> np.ndarray:
    return v / np.linalg.norm(v)

def rotation_matrix_to_quaternion(R: np.ndarray):
    qw = np.sqrt(1 + R[0,0] + R[1,1] + R[2,2]) / 2
    qx = (R[2,1] - R[1,2]) / (4*qw)
    qy = (R[0,2] - R[2,0]) / (4*qw)
    qz = (R[1,0] - R[0,1]) / (4*qw)
    return [qw, qx, qy, qz]

# —— 5. 主循环 —— 
cv2.namedWindow("Hand Tracking", cv2.WINDOW_AUTOSIZE)
try:
    while True:
        # 5.1 获取对齐帧
        frames = pipeline.wait_for_frames()
        aligned = align.process(frames)
        depth_frame = aligned.get_depth_frame()
        color_frame = aligned.get_color_frame()
        # 如果任意一帧为空，就 skip
        if not depth_frame or not color_frame:
            print("Warning: empty frame")
            continue

        # 转成 numpy
        color = np.asanyarray(color_frame.get_data())
        display = color.copy()  # 绘制用的副本

        # 5.2 MediaPipe 检测
        rgb = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb)
        if not result.multi_hand_landmarks:
            continue

        lm = result.multi_hand_landmarks[0]
        #mp_drawing.draw_landmarks(color, lm, mp_hands.HAND_CONNECTIONS)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(
                    display,
                    hand_landmarks,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color=(0,255,0), thickness=2, circle_radius=2),
                    mp_drawing.DrawingSpec(color=(255,0,0), thickness=2))
        

        # 最后展示绘制结果
        cv2.imshow("Hand Tracking", display)
        key = cv2.waitKey(1)
        if key == ord('q'):
            break


        # 5.3 计算手腕像素深度
        u = int(lm.landmark[mp_hands.HandLandmark.WRIST].x * intr.width)
        v = int(lm.landmark[mp_hands.HandLandmark.WRIST].y * intr.height)
        # 边界检查
        u = np.clip(u, 0, intr.width-1)
        v = np.clip(v, 0, intr.height-1)
        D = depth_frame.get_distance(u, v)

        # 5.4 构建局部坐标系（z=腕→指根，x=腕→拇指投影，y=叉乘）
        W = np.array([lm.landmark[0].x, lm.landmark[0].y, lm.landmark[0].z])
        P_idx = np.array([lm.landmark[5].x, lm.landmark[5].y, lm.landmark[5].z])
        P_thu = np.array([lm.landmark[2].x, lm.landmark[2].y, lm.landmark[2].z])

        z_axis = normalize(P_idx - W)
        temp = P_thu - W
        x_axis = normalize(temp - np.dot(temp, z_axis)*z_axis)
        y_axis = np.cross(z_axis, x_axis)
        R_hand = np.stack([x_axis, y_axis, z_axis], axis=1)

        # 5.5 转到基座系、四元数→欧拉
        R_base = R_rc.dot(R_hand)
        quat = rotation_matrix_to_quaternion(R_base)
        eul = algo.rm_algo_quaternion2euler(quat)  # [rx, ry, rz]

        # 5.6 下发跟随指令（位置 pos0 不变）
        target = pos0 + eul
        print(f"{target}")
        ret = arm.rm_movep_follow(target)
        # if ret != 0:
        #     print("FollowError:", ret)


finally:
    pipeline.stop()
    hands.close()
    arm.rm_delete_robot_arm()
    cv2.destroyAllWindows()


current c api version:  1.1.3


I0000 00:00:1765778102.177935  275529 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1765778102.235072  275638 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 580.95.05), renderer: NVIDIA GeForce RTX 4090/PCIe/SSE2
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1765778102.255442  275612 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765778102.271105  275635 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765778105.094687  275631 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


[-0.263033, -0.014339, 0.318728, 2.8650097846984863, 0.03764599561691284, -1.6343283653259277]
[-0.263033, -0.014339, 0.318728, 2.7111923694610596, 0.07433231920003891, -1.7928417921066284]
[-0.263033, -0.014339, 0.318728, 2.751530408859253, 0.13842478394508362, -1.8617609739303589]
[-0.263033, -0.014339, 0.318728, 2.666343927383423, 0.1981131136417389, -1.7233424186706543]
[-0.263033, -0.014339, 0.318728, 2.70123291015625, 0.22923210263252258, -1.7108900547027588]
[-0.263033, -0.014339, 0.318728, 2.717816114425659, 0.21432699263095856, -1.7621947526931763]
[-0.263033, -0.014339, 0.318728, 2.7124242782592773, 0.22729308903217316, -1.7381715774536133]
[-0.263033, -0.014339, 0.318728, 2.6852540969848633, 0.21490751206874847, -1.7257362604141235]
[-0.263033, -0.014339, 0.318728, 2.6873342990875244, 0.20208574831485748, -1.6897475719451904]
[-0.263033, -0.014339, 0.318728, 2.6937243938446045, 0.18006162345409393, -1.6938709020614624]
[-0.263033, -0.014339, 0.318728, 2.696796417236328, 0.17

/tmp/ipykernel_275529/1922323750.py:54: RuntimeWarning: invalid value encountered in sqrt
  qw = np.sqrt(1 + R[0,0] + R[1,1] + R[2,2]) / 2


[-0.263033, -0.014339, 0.318728, 3.0056233406066895, 0.24428409337997437, -1.5960819721221924]
[-0.263033, -0.014339, 0.318728, 3.0017266273498535, 0.2645259201526642, -1.6065372228622437]
[-0.263033, -0.014339, 0.318728, 2.9784631729125977, 0.27135583758354187, -1.5878061056137085]
[-0.263033, -0.014339, 0.318728, 2.982898473739624, 0.29985421895980835, -1.5465847253799438]
[-0.263033, -0.014339, 0.318728, 3.006864547729492, 0.31980815529823303, -1.5191172361373901]
[-0.263033, -0.014339, 0.318728, 2.973809242248535, 0.3249695599079132, -1.5381617546081543]
[-0.263033, -0.014339, 0.318728, 2.963188409805298, 0.3352411985397339, -1.5409467220306396]
[-0.263033, -0.014339, 0.318728, 2.940886974334717, 0.3362089693546295, -1.553580403327942]
[-0.263033, -0.014339, 0.318728, 2.9387357234954834, 0.3509848415851593, -1.5150965452194214]
[-0.263033, -0.014339, 0.318728, 2.938056230545044, 0.3574155271053314, -1.522421956062317]
[-0.263033, -0.014339, 0.318728, 2.9277210235595703, 0.352690070

: 

In [ ]:
'''3D 映射 (PBVS 增量映射) 笛卡尔空间'''
import time
import cv2
import numpy as np
import mediapipe as mp
import pyrealsense2 as rs
from Robotic_Arm.rm_robot_interface import *

# —— 启动计时器 —— 
start_time = time.time()

# —— 1. 初始化 RealSense —— 
pipeline = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)    
cfg.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)      
profile = pipeline.start(cfg)
align = rs.align(rs.stream.color)

# 拿内参
color_stream = profile.get_stream(rs.stream.color)
intr = color_stream.as_video_stream_profile().get_intrinsics()
fx, fy, cx, cy = intr.fx, intr.fy, intr.ppx, intr.ppy

# —— 2. 初始化机械臂 —— 
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
arm_handle = arm.rm_create_robot_arm("192.168.1.18", 8080)
arm_model = rm_robot_arm_model_e.RM_MODEL_RM_65_E
force_type = rm_force_type_e.RM_MODEL_RM_B_E
algo = Algo(arm_model=arm_model, force_type=force_type)


# 固定末端起始位置（基座系下）
arm.rm_movej([2.3, 12.26, 85.40, 1.52, 81.97, -86.50], 50, 0, 0, 1)
time.sleep(0.5)
pos0 = arm.rm_get_current_arm_state()[1]['pose'][:3]
# —— 3. 手–眼标定旋转矩阵 —— 
q = np.array([-0.464607, 0.552742, -0.525271, 0.450230])
w, x, y, z = q
R_rc = np.array([
    [1-2*(y*y+z*z),   2*(x*y - z*w),   2*(x*z + y*w)],
    [2*(x*y + z*w),   1-2*(x*x+z*z),   2*(y*z - x*w)],
    [2*(x*z - y*w),   2*(y*z + x*w),   1-2*(x*x+y*y)]
], dtype=np.float32)

# —— 4. MediaPipe Hands —— 
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.9,
                       min_tracking_confidence=0.9,
                       max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

def normalize(v: np.ndarray) -> np.ndarray:
    return v / np.linalg.norm(v)

def rotation_matrix_to_quaternion(R: np.ndarray):
    qw = np.sqrt(1 + R[0,0] + R[1,1] + R[2,2]) / 2
    qx = (R[2,1] - R[1,2]) / (4*qw)
    qy = (R[0,2] - R[2,0]) / (4*qw)
    qz = (R[1,0] - R[0,1]) / (4*qw)
    return [qw, qx, qy, qz]

# —— 5. 采集手零位 Pc0 —— 
Pc0 = None
scale = 0.7  # 映射灵敏度，可调

print("请将手腕移到'零位'并保持不动,4 秒后开始零点标定...")
while Pc0 is None:
    # 检查是否过了 4 秒
    if time.time() - start_time < 4.0:
        time.sleep(0.1)
        continue

    frames = pipeline.wait_for_frames()
    aligned = align.process(frames)
    d = aligned.get_depth_frame(); c = aligned.get_color_frame()
    if not d or not c: continue

    color = np.asanyarray(c.get_data())
    rgb = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
    res = hands.process(rgb)
    if not res.multi_hand_landmarks: 
        cv2.imshow("Init Zero Pose (Q退出)", color); cv2.waitKey(1)
        continue

    lm = res.multi_hand_landmarks[0]
    u = int(lm.landmark[mp_hands.HandLandmark.WRIST].x * intr.width)
    v = int(lm.landmark[mp_hands.HandLandmark.WRIST].y * intr.height)
    u,v = np.clip(u,0,intr.width-1), np.clip(v,0,intr.height-1)
    D = d.get_distance(u,v)
    Xc = (u-cx)*D/fx; Yc = (v-cy)*D/fy; Zc = D
    Pc0 = np.array([Xc, Yc, Zc], dtype=np.float32)
    print("零位 Pc0 =", Pc0)
    cv2.destroyAllWindows()

# —— 6. 主循环：增量映射 + 姿态控制 —— 
cv2.namedWindow("Hand→Arm PBVS", cv2.WINDOW_AUTOSIZE)
try:
    while True:
        frames = pipeline.wait_for_frames()
        aligned = align.process(frames)
        depth_frame = aligned.get_depth_frame()
        color_frame = aligned.get_color_frame()
        if not depth_frame or not color_frame:
            continue

        color = np.asanyarray(color_frame.get_data())
        display = color.copy()
        rgb = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
        res = hands.process(rgb)
        if not res.multi_hand_landmarks:
            cv2.imshow("Hand→Arm PBVS", display)
            if cv2.waitKey(1)==ord('q'): break
            continue

        lm = res.multi_hand_landmarks[0]
        mp_drawing.draw_landmarks(display, lm, mp_hands.HAND_CONNECTIONS)

        # ——— 6.1 反投影当前手腕 Pc ———
        u = int(lm.landmark[mp_hands.HandLandmark.WRIST].x * intr.width)
        v = int(lm.landmark[mp_hands.HandLandmark.WRIST].y * intr.height)
        u,v = np.clip(u,0,intr.width-1), np.clip(v,0,intr.height-1)
        D = depth_frame.get_distance(u,v)
        Pc = np.array([(u-cx)*D/fx, (v-cy)*D/fy, D], dtype=np.float32)

        # ——— 6.2 增量映射到基座系 ΔPr ———
        deltaPc = Pc - Pc0                     # 相机系增量
        deltaPr = R_rc.dot(deltaPc) * scale    # 基座系增量

        # # ——— 6.3 姿态计算 ———
        # W = np.array([lm.landmark[0].x, lm.landmark[0].y, lm.landmark[0].z])
        # P_idx = np.array([lm.landmark[5].x, lm.landmark[5].y, lm.landmark[5].z])
        # P_thu = np.array([lm.landmark[2].x, lm.landmark[2].y, lm.landmark[2].z])
        # z_axis = normalize(P_idx - W)
        # tmp = P_thu - W
        # x_axis = normalize(tmp - np.dot(tmp, z_axis)*z_axis)
        # y_axis = np.cross(z_axis, x_axis)
        # R_hand = np.stack([x_axis, y_axis, z_axis], axis=1)
        # R_base = R_rc.dot(R_hand)
        # quat = rotation_matrix_to_quaternion(R_base)
        # eul = algo.rm_algo_quaternion2euler(quat)  # [rx, ry, rz]

        eul_fixed = [3.146, 0.026, 1.546]

        # ——— 6.4 目标末端位姿 = 起始位置 + ΔPr + 当前姿态 ———
        target_pos = pos0 + deltaPr
        target_pose = np.concatenate([target_pos, eul_fixed]).tolist()  # [x,y,z,rx,ry,rz]

        print(f"目标位姿态:{target_pose}")

        # ——— 6.5 下发跟随指令 ———
        ret = arm.rm_movep_follow(target_pose)
        # if ret != 0:
        #     print("FollowError:", ret)

        # ——— 6.6 显示 & 退出 ———
        cv2.imshow("Hand→Arm PBVS", display)
        if cv2.waitKey(1) == ord('q'):
            break

finally:
    pipeline.stop()
    hands.close()
    arm.rm_delete_robot_arm()
    cv2.destroyAllWindows()


current c api version:  1.1.3


I0000 00:00:1765777684.126507  271035 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1765777684.172062  275013 gl_context.cc:369] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 580.95.05), renderer: NVIDIA GeForce RTX 4090/PCIe/SSE2
W0000 00:00:1765777684.187514  274984 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1765777684.202028  275006 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


请将手腕移到'零位'并保持不动,4 秒后开始零点标定...
零位 Pc0 = [ 0.03040287 -0.02989856  0.51600003]
目标位姿态:[-0.26233592098959163, -0.014871848986212164, 0.3197019718455523, 3.146, 0.026, 1.546]
目标位姿态:[-0.2645231739073843, -0.014841619244970381, 0.31936442334981635, 3.146, 0.026, 1.546]
目标位姿态:[-0.2638245028132424, -0.01485478401343152, 0.3192919113308564, 3.146, 0.026, 1.546]
目标位姿态:[-0.26373353297038377, -0.014845558273799717, 0.3198448418507725, 3.146, 0.026, 1.546]
目标位姿态:[-0.26443238101581484, -0.014832411491505802, 0.3199162784831971, 3.146, 0.026, 1.546]
目标位姿态:[-0.264250795232676, -0.014813995926368981, 0.32101998863354325, 3.146, 0.026, 1.546]
目标位姿态:[-0.2655586739923656, -0.014778605115560815, 0.3217082714064419, 3.146, 0.026, 1.546]
目标位姿态:[-0.26546823465412855, -0.014769433159807697, 0.3222579780368507, 3.146, 0.026, 1.546]
目标位姿态:[-0.26546823465412855, -0.014769433159807697, 0.3222579780368507, 3.146, 0.026, 1.546]
目标位姿态:[-0.2661012322563529, -0.014192442576012574, 0.32286840410515666, 3.146, 0.026, 1.54

KeyboardInterrupt: 

In [ ]:
'''分析雅可比矩阵最小奇异值, 判断机器人是否处于奇异状态（笛卡尔空间）'''
import time
import cv2
import numpy as np
import mediapipe as mp
import pyrealsense2 as rs
from Robotic_Arm.rm_robot_interface import (
    RoboticArm, Algo, rm_thread_mode_e
)

def normalize(v: np.ndarray) -> np.ndarray:
    return v / np.linalg.norm(v)

def rotation_matrix_to_quaternion(R: np.ndarray):
    qw = np.sqrt(1 + R[0,0] + R[1,1] + R[2,2]) / 2
    qx = (R[2,1] - R[1,2]) / (4*qw)
    qy = (R[0,2] - R[2,0]) / (4*qw)
    qz = (R[1,0] - R[0,1]) / (4*qw)
    return [qw, qx, qy, qz]

# 1. 初始化 RealSense
pipeline = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
cfg.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
profile = pipeline.start(cfg)
align = rs.align(rs.stream.color)

# 相机内参
color_stream = profile.get_stream(rs.stream.color)
intr = color_stream.as_video_stream_profile().get_intrinsics()
fx, fy, cx, cy = intr.fx, intr.fy, intr.ppx, intr.ppy

# 2. 初始化机械臂 & 算法
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
arm.rm_create_robot_arm("169.254.128.19", 8080)
arm_model = rm_robot_arm_model_e.RM_MODEL_RM_75_E  # RM_65机械臂
force_type = rm_force_type_e.RM_MODEL_RM_B_E  # 标准版
algo = Algo(arm_model, force_type)

# 末端初始位置 & 手眼标定旋转
pos0 = np.array([0.30713, 0.006135, 0.357776], dtype=np.float32)
# 示例四元数
q = np.array([0.3553804278227774, 0.18731548964299938, 0.8637697797439028, -0.30417006174420397])
w,x,y,z = q
R_rc = np.array([
    [1-2*(y*y+z*z),   2*(x*y - z*w),   2*(x*z + y*w)],
    [2*(x*y + z*w),   1-2*(x*x+z*z),   2*(y*z - x*w)],
    [2*(x*z - y*w),   2*(y*z + x*w),   1-2*(x*x+y*y)]
], dtype=np.float32)

# 3. MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.8,
                       min_tracking_confidence=0.8,
                       max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# 4. 零位标定
Pc0 = None
start_time = time.time()
print("请将手腕移到“零位”并保持不动，4 秒后开始标定…")
while Pc0 is None:
    if time.time() - start_time < 2.0:
        time.sleep(0.1); continue
    frames = pipeline.wait_for_frames()
    aligned = align.process(frames)
    df, cf = aligned.get_depth_frame(), aligned.get_color_frame()
    if not df or not cf: continue
    color = np.asanyarray(cf.get_data())
    rgb   = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
    res   = hands.process(rgb)
    if not res.multi_hand_landmarks:
        cv2.imshow("Zero Pose", color); cv2.waitKey(1); continue
    lm = res.multi_hand_landmarks[0]
    u = int(lm.landmark[mp_hands.HandLandmark.WRIST].x * intr.width)
    v = int(lm.landmark[mp_hands.HandLandmark.WRIST].y * intr.height)
    u,v = np.clip(u,0,intr.width-1), np.clip(v,0,intr.height-1)
    D = df.get_distance(u,v)
    Pc0 = np.array([(u-cx)*D/fx, (v-cy)*D/fy, D], dtype=np.float32)
    print("标定零位 Pc0 =", Pc0)
    cv2.destroyAllWindows()

# 5. 主循环：PBVS + 奇异点检测 + 跳过
cv2.namedWindow("PBVS with Singularity Check", cv2.WINDOW_AUTOSIZE)
while True:
    frames = pipeline.wait_for_frames()
    aligned = align.process(frames)
    df, cf = aligned.get_depth_frame(), aligned.get_color_frame()
    if not df or not cf:
        continue

    color   = np.asanyarray(cf.get_data())
    display = color.copy()
    rgb     = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
    res     = hands.process(rgb)

    if res.multi_hand_landmarks:
        lm = res.multi_hand_landmarks[0]
        mp_drawing.draw_landmarks(display, lm, mp_hands.HAND_CONNECTIONS)

        # 5.1 反投影手腕 Pc
        u = int(lm.landmark[mp_hands.HandLandmark.WRIST].x * intr.width)
        v = int(lm.landmark[mp_hands.HandLandmark.WRIST].y * intr.height)
        u,v = np.clip(u,0,intr.width-1), np.clip(v,0,intr.height-1)
        D = df.get_distance(u,v)
        Pc = np.array([(u-cx)*D/fx, (v-cy)*D/fy, D], dtype=np.float32)

        # 5.2 增量映射 ΔPr
        deltaPr = R_rc.dot(Pc - Pc0)

        # 5.3 姿态计算
        W    = np.array([lm.landmark[0].x, lm.landmark[0].y, lm.landmark[0].z])
        I    = np.array([lm.landmark[5].x, lm.landmark[5].y, lm.landmark[5].z])
        T    = np.array([lm.landmark[2].x, lm.landmark[2].y, lm.landmark[2].z])
        z_ax = normalize(I - W)
        tmp  = T - W
        x_ax = normalize(tmp - np.dot(tmp, z_ax)*z_ax)
        y_ax = np.cross(z_ax, x_ax)
        R_hand = np.stack([x_ax, y_ax, z_ax], axis=1)
        R_base = R_rc.dot(R_hand)
        quat   = rotation_matrix_to_quaternion(R_base)
        eul    = algo.rm_algo_quaternion2euler(quat)  # [rx, ry, rz]

        # 5.4 目标位姿
        target_pos  = pos0 + deltaPr
        target_pose = [*target_pos.tolist(), *eul]

        # —— 奇异点检测 —— 
        # 读取当前关节角度
        # 1) 构造逆解参数并求解目标关节角度
        q_current = arm.rm_get_joint_degree()[1]
        #print(q_current)
        params = rm_inverse_kinematics_params_t(q_current, target_pose, 1)
        ret_ik, q_target = algo.rm_algo_inverse_kinematics(params)
        if ret_ik != 0:
            print("code:", ret_ik)
            continue
        # 2) 对目标关节角度做奇异点检测
        sing = algo.rm_algo_universal_singularity_analyse(q_target, 0.01)
        if sing == -1:
            print("目标关节角度处于奇异区，跳过本次运动:", q_target)
            continue
        elif sing < 0:
            print("奇异点检测出错，code:", sing)

        # 5.5 下发笛卡尔运动
        ret = arm.rm_movep_follow(q_target)
        if ret != 0:
            print("rm_movej_follow 失败，code:", ret)

    cv2.imshow("PBVS with Singularity Check", display)
    if cv2.waitKey(1) == ord('q'):
        break

pipeline.stop()
hands.close()
arm.rm_delete_robot_arm()
cv2.destroyAllWindows()


In [ ]:
'''用关节空间跟随运动rm_movej_follow()避免奇异点，改为四元数逆解并做姿态/位置平滑以减抖'''
import time
import cv2
import numpy as np
import mediapipe as mp
import pyrealsense2 as rs
from collections import deque
from Robotic_Arm.rm_robot_interface import *

JOINT_QUEUE_LEN = 4    # 平衡关节抖动（滑动平均）
joint_queue = deque(maxlen=JOINT_QUEUE_LEN)

# 归一化
def normalize(v: np.ndarray) -> np.ndarray:
    return v / np.linalg.norm(v)

# 转四元数
def rotation_matrix_to_quaternion(R: np.ndarray):
    qw = np.sqrt(1 + R[0,0] + R[1,1] + R[2,2]) / 2
    qx = (R[2,1] - R[1,2]) / (4*qw)
    qy = (R[0,2] - R[2,0]) / (4*qw)
    qz = (R[1,0] - R[0,1]) / (4*qw)
    return [qw, qx, qy, qz]

def quat_hemisphere_consistency(q_ref: np.ndarray, q: np.ndarray) -> np.ndarray:
    # 使相邻四元数位于同一半球，避免slerp走长弧
    if np.dot(q_ref, q) < 0.0:
        return -q
    return q

def quat_slerp(q0: np.ndarray, q1: np.ndarray, t: float) -> np.ndarray:
    # 球面线性插值，q为[w,x,y,z]
    dot = float(np.clip(np.dot(q0, q1), -1.0, 1.0))
    if dot > 0.9995:
        # 夹角很小时退化为线性再归一化
        res = q0 + t * (q1 - q0)
        return res / np.linalg.norm(res)
    theta_0 = np.arccos(dot)
    sin_theta_0 = np.sin(theta_0)
    theta = theta_0 * t
    s0 = np.sin(theta_0 - theta) / sin_theta_0
    s1 = np.sin(theta) / sin_theta_0
    res = s0 * q0 + s1 * q1
    return res / np.linalg.norm(res)

# 1. 初始化 RealSense
pipeline = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
cfg.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
profile = pipeline.start(cfg)
align = rs.align(rs.stream.color)

# 相机内参
color_stream = profile.get_stream(rs.stream.color)
intr = color_stream.as_video_stream_profile().get_intrinsics()
fx, fy, cx, cy = intr.fx, intr.fy, intr.ppx, intr.ppy

# 2. 初始化机械臂 & 算法
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
arm.rm_create_robot_arm("169.254.128.19", 8080)
algo = Algo(arm_model=1, force_type=0)

# 3. 运动到初始末端位置 
arm.rm_movej([1.1269999742507935, 30.274999618530273, -4.90500020980835, 85.21499633789062, 21.239999771118164, 19.764999389648438, -16.999000549316406], 20, 0, 0, 1)
# 读取初始位置
pose0 = arm.rm_get_current_arm_state()[1]['pose'][:3]      # 只取x y z

# ——— 平滑参数 ———
QUAT_ALPHA = 0.2   # 四元数slerp系数，越小越稳
POS_ALPHA  = 0.3   # 位置EMA系数，越小越稳
prev_quat  = None
pos_smooth = np.array(pose0, dtype=np.float32)

# 手–眼标定旋转
q = np.array([0.2659364856156465, 0.2742003991705386, 0.7589529368202654, -0.5273351556695332])
w, x, y, z = q
R_rc = np.array([
    [1-2*(y*y+z*z),   2*(x*y - z*w),   2*(x*z + y*w)],
    [2*(x*y + z*w),   1-2*(x*x+z*z),   2*(y*z - x*w)],
    [2*(x*z - y*w),   2*(y*z + x*w),   1-2*(x*x+y*y)]
], dtype=np.float32)

# 4. MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.9,
                       min_tracking_confidence=0.9,
                       max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# 5. 零位标定
Pc0 = None
scale = 1.2  # 映射灵敏度，可调
start_time = time.time()
print("请将手腕移到“零位”并保持不动, 2秒后开始标定…")
while Pc0 is None:
    if time.time() - start_time < 2.0:
        time.sleep(0.1)
        continue
    frames = pipeline.wait_for_frames()
    aligned = align.process(frames)
    df, cf = aligned.get_depth_frame(), aligned.get_color_frame()
    if not df or not cf:
        continue

    color = np.asanyarray(cf.get_data())
    rgb   = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
    res   = hands.process(rgb)
    if not res.multi_hand_landmarks:
        cv2.imshow("Init Zero Pose (Q退出)", color)
        cv2.waitKey(1)
        continue

    lm = res.multi_hand_landmarks[0]
    u = int(lm.landmark[mp_hands.HandLandmark.WRIST].x * intr.width)
    v = int(lm.landmark[mp_hands.HandLandmark.WRIST].y * intr.height)
    u, v = np.clip(u,0,intr.width-1), np.clip(v,0,intr.height-1)
    D = df.get_distance(u, v)
    Pc0 = np.array([(u-cx)*D/fx, (v-cy)*D/fy, D], dtype=np.float32)
    print("标定零位 Pc0 =", Pc0)
    cv2.destroyAllWindows()

# 6. 主循环：增量映射 + 姿态(四元数) + 逆解 + rm_movej_follow
cv2.namedWindow("Hand→Arm Joint PBVS", cv2.WINDOW_NORMAL)
cv2.setWindowProperty("Hand→Arm Joint PBVS", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
try:
    while True:
        frames  = pipeline.wait_for_frames()
        aligned = align.process(frames)
        df, cf = aligned.get_depth_frame(), aligned.get_color_frame()
        if not df or not cf:
            continue

        color   = np.asanyarray(cf.get_data())
        display = color.copy()
        rgb     = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
        res     = hands.process(rgb)

        if res.multi_hand_landmarks:
            lm = res.multi_hand_landmarks[0]
            mp_drawing.draw_landmarks(display, lm, mp_hands.HAND_CONNECTIONS)

            # 6.1 反投影mediapipe的关键点到相机坐标系下的真实 3D 坐标
            def unproject(lm):
                u = int(lm.x * intr.width)
                v = int(lm.y * intr.height)
                u, v = np.clip(u,0,intr.width-1), np.clip(v,0,intr.height-1)
                D = df.get_distance(u, v)
                X = (u - cx) * D / fx
                Y = (v - cy) * D / fy
                Z = D
                return np.array([X, Y, Z], dtype=np.float32)
            
            # 手腕、无名指根（RING_FINGER_MCP）、拇指根（THUMB_MCP）
            Wc  = unproject(lm.landmark[mp_hands.HandLandmark.WRIST])
            Fc  = unproject(lm.landmark[mp_hands.HandLandmark.RING_FINGER_MCP])
            Tc  = unproject(lm.landmark[mp_hands.HandLandmark.THUMB_MCP])

            # 6.2 增量映射 ΔPr (相机→基座)
            deltaPr = R_rc.dot(Wc - Pc0) * scale

            # 6.3 姿态计算(用真实 3D 坐标构建手部局部坐标系) 
            z_ax = normalize(Fc - Wc)          # Z轴：腕→无名指根(因为mediapipe里手腕点到无名指根点是近似垂直的)
            tmp  = Tc - Wc                     # 腕→拇指根
            x_ax = normalize(tmp - np.dot(tmp, z_ax) * z_ax)   # 去掉在 z_ax 上的分量
            y_ax = np.cross(z_ax, x_ax)

            R_hand = np.stack([x_ax, y_ax, z_ax], axis=1)
            R_base = R_rc.dot(R_hand)

            # 当前四元数（w,x,y,z）并归一化
            q_now = np.array(rotation_matrix_to_quaternion(R_base), dtype=np.float64)
            q_now /= np.linalg.norm(q_now)

            # 四元数半球一致性 + SLERP 平滑
            if prev_quat is None:
                prev_quat = q_now.copy()
            q_now = quat_hemisphere_consistency(prev_quat, q_now)
            q_smooth = quat_slerp(prev_quat, q_now, QUAT_ALPHA)
            prev_quat = q_smooth.copy()

            # 6.4 目标末端位置EMA平滑
            target_pos = np.array(pose0, dtype=np.float32) + deltaPr
            pos_smooth = POS_ALPHA * target_pos + (1.0 - POS_ALPHA) * pos_smooth

            # 6.5 目标末端位姿(位置+四元数) → 逆解(flag=0使用四元数)
            target_pose = np.concatenate([pos_smooth, q_smooth]).tolist()
            q_current = arm.rm_get_joint_degree()[1]
            
            # 6.5 构造逆解参数并调用逆解
            params = rm_inverse_kinematics_params_t(q_current, target_pose,0)
            ret_ik, joints = algo.rm_algo_inverse_kinematics(params)
            #print(joints)
            if ret_ik == 0:
                # 6.6 关节滑动平均再下发
                joint_queue.append(joints)
                avg_joints = np.mean(joint_queue, axis=0).tolist()
                # 6.6 平滑后关节空间跟随下发
                arm.rm_movej_follow(avg_joints)
            else:
                print("code:", ret_ik)

        cv2.imshow("Hand→Arm Joint PBVS", display)
        if cv2.waitKey(1) == ord('q'):
            break

finally:
    pipeline.stop()
    hands.close()
    arm.rm_delete_robot_arm()
    cv2.destroyAllWindows()   

current c api version:  1.1.0
请将手腕移到“零位”并保持不动, 2秒后开始标定…


W0000 00:00:1758610327.723741  170558 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1758610327.728958  170558 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


标定零位 Pc0 = [-0.03331661 -0.01863657  0.81200004]


In [ ]:
'''用关节空间跟随运动rm_movej_follow()避免奇异点，再用卡尔曼滤波平滑'''
import time
import cv2
import numpy as np
import mediapipe as mp
import pyrealsense2 as rs
from collections import deque
from Robotic_Arm.rm_robot_interface import *

# 定义卡尔曼滤波器类 —— 
class Kalman1D:
    def __init__(self, q=1e-5, r=1e-2):
        # 状态向量 x = [angle, velocity]^T
        self.A = np.array([[1, 1],
                           [0, 1]], dtype=float)   # 状态转移矩阵
        self.H = np.array([[1, 0]], dtype=float)    # 观测矩阵
        self.Q = np.eye(2) * q                       # 过程噪声协方差
        self.R = np.array([[r]], dtype=float)        # 观测噪声协方差
        self.P = np.eye(2)                           # 后验估计协方差
        self.x = np.zeros((2, 1), dtype=float)       # 后验状态

    def update(self, measurement: float) -> float:
        # — predict —
        self.x = self.A @ self.x
        self.P = self.A @ self.P @ self.A.T + self.Q

        # — update —
        z = np.array([[measurement]], dtype=float)
        y = z - self.H @ self.x
        S = self.H @ self.P @ self.H.T + self.R
        K = self.P @ self.H.T @ np.linalg.inv(S)
        self.x = self.x + K @ y
        self.P = (np.eye(2) - K @ self.H) @ self.P

        return float(self.x[0, 0])


# 归一化
def normalize(v: np.ndarray) -> np.ndarray:
    return v / np.linalg.norm(v)

# 转四元数
def rotation_matrix_to_quaternion(R: np.ndarray):
    qw = np.sqrt(1 + R[0,0] + R[1,1] + R[2,2]) / 2
    qx = (R[2,1] - R[1,2]) / (4*qw)
    qy = (R[0,2] - R[2,0]) / (4*qw)
    qz = (R[1,0] - R[0,1]) / (4*qw)
    return [qw, qx, qy, qz]

# 1. 初始化 RealSense
pipeline = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
cfg.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
profile = pipeline.start(cfg)
align = rs.align(rs.stream.color)

# 相机内参
color_stream = profile.get_stream(rs.stream.color)
intr = color_stream.as_video_stream_profile().get_intrinsics()
fx, fy, cx, cy = intr.fx, intr.fy, intr.ppx, intr.ppy

# 2. 初始化机械臂 & 算法
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
arm.rm_create_robot_arm("169.254.128.19", 8080)
algo = Algo(arm_model=1, force_type=0)

# 3. 运动到初始末端位置 
arm.rm_movej([1.1269999742507935, 30.274999618530273, -4.90500020980835, 85.21499633789062, 21.239999771118164, 19.764999389648438, -16.999000549316406], 20, 0, 0, 1)
# 读取初始位置
pose0 = arm.rm_get_current_arm_state()[1]['pose'][:3]      # 只取x y z

# 手–眼标定旋转
q = np.array([0.3553804278227774, 0.18731548964299938, 0.8637697797439028, -0.30417006174420397])
w, x, y, z = q
R_rc = np.array([
    [1-2*(y*y+z*z),   2*(x*y - z*w),   2*(x*z + y*w)],
    [2*(x*y + z*w),   1-2*(x*x+z*z),   2*(y*z - x*w)],
    [2*(x*z - y*w),   2*(y*z + x*w),   1-2*(x*x+y*y)]
], dtype=np.float32)

# 4. MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.9,
                       min_tracking_confidence=0.9,
                       max_num_hands=1)
mp_drawing = mp.solutions.drawing_utils

# 5. 零位标定
Pc0 = None
scale = 1.2  # 映射灵敏度，可调
start_time = time.time()
print("请将手腕移到“零位”并保持不动, 2秒后开始标定…")
while Pc0 is None:
    if time.time() - start_time < 2.0:
        time.sleep(0.1)
        continue
    frames = pipeline.wait_for_frames()
    aligned = align.process(frames)
    df, cf = aligned.get_depth_frame(), aligned.get_color_frame()
    if not df or not cf:
        continue

    color = np.asanyarray(cf.get_data())
    rgb   = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
    res   = hands.process(rgb)
    if not res.multi_hand_landmarks:
        cv2.imshow("Init Zero Pose (Q退出)", color)
        cv2.waitKey(1)
        continue

    lm = res.multi_hand_landmarks[0]
    u = int(lm.landmark[mp_hands.HandLandmark.WRIST].x * intr.width)
    v = int(lm.landmark[mp_hands.HandLandmark.WRIST].y * intr.height)
    u, v = np.clip(u,0,intr.width-1), np.clip(v,0,intr.height-1)
    D = df.get_distance(u, v)
    Pc0 = np.array([(u-cx)*D/fx, (v-cy)*D/fy, D], dtype=np.float32)
    print("标定零位 Pc0 =", Pc0)
    cv2.destroyAllWindows()

# —— 创建一组关节滤波器 —— 
NUM_JOINTS = 7
joint_filters = [Kalman1D(q=1e-8, r=1e-5) for _ in range(NUM_JOINTS)]  #q（过程噪声方差）越小，对突变响应越快，r（测量噪声方差）越，平滑效果越强但响应更慢

# 6. 主循环：增量映射 + 姿态 + 逆解 + rm_movej_follow
cv2.namedWindow("Hand→Arm Joint PBVS", cv2.WINDOW_AUTOSIZE)
try:
    while True:
        frames  = pipeline.wait_for_frames()
        aligned = align.process(frames)
        df, cf = aligned.get_depth_frame(), aligned.get_color_frame()
        if not df or not cf:
            continue

        color   = np.asanyarray(cf.get_data())
        display = color.copy()
        rgb     = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
        res     = hands.process(rgb)

        if res.multi_hand_landmarks:
            lm = res.multi_hand_landmarks[0]
            mp_drawing.draw_landmarks(display, lm, mp_hands.HAND_CONNECTIONS)

            # 6.1 反投影mediapipe的关键点到相机坐标系下的真实 3D 坐标
            def unproject(lm):
                u = int(lm.x * intr.width)
                v = int(lm.y * intr.height)
                u, v = np.clip(u,0,intr.width-1), np.clip(v,0,intr.height-1)
                D = df.get_distance(u, v)    # 深度（米）
                X = (u - cx) * D / fx
                Y = (v - cy) * D / fy
                Z = D
                return np.array([X, Y, Z], dtype=np.float32)
            
            # 手腕、无名指根（RING_FINGER_MCP）、拇指根（THUMB_MCP）
            Wc  = unproject(lm.landmark[mp_hands.HandLandmark.WRIST])
            Fc  = unproject(lm.landmark[mp_hands.HandLandmark.RING_FINGER_MCP])
            Tc  = unproject(lm.landmark[mp_hands.HandLandmark.THUMB_MCP]) 

            # 6.2 增量映射 ΔPr
            deltaPr = R_rc.dot(Wc - Pc0) * scale

            # 6.3 姿态计算(用真实 3D 坐标构建手部局部坐标系) 
            z_ax = normalize(Fc - Wc)          # Z轴：腕→无名指根(因为mediapipe里手腕点到无名指根点是近似垂直的)
            tmp  = Tc - Wc                     # 腕→拇指根
            x_ax = normalize(tmp - np.dot(tmp, z_ax)*z_ax)   # 去掉在 z_ax 上的分量
            y_ax = np.cross(z_ax, x_ax)

            R_hand = np.stack([x_ax, y_ax, z_ax], axis=1)
            R_base = R_rc.dot(R_hand)

            quat   = rotation_matrix_to_quaternion(R_base)
            # ---- 四元数归一化 ----
            qv = np.array(quat, dtype=np.float64)
            qv /= np.linalg.norm(qv)
            quat = qv.tolist()
            eul    = algo.rm_algo_quaternion2euler(quat)  # [rx, ry, rz]

            # 6.4 目标末端位姿
            target_pos  = pose0 + deltaPr
            target_pose = np.concatenate([target_pos, eul]).tolist()
            q_current = arm.rm_get_joint_degree()[1]
            
            # 6.5 构造逆解参数并调用逆解
            params = rm_inverse_kinematics_params_t(q_current, target_p ose,1)
            ret_ik, joints = algo.rm_algo_inverse_kinematics(params)
            #print(joints)
            if ret_ik == 0:
                # joints 是逆解得到的原始目标关节角度列表，对它做卡尔曼滤波平滑
                smoothed = [joint_filters[i].update(joints[i]) for i in range(NUM_JOINTS)]

                # 6.6 平滑后关节空间跟随下发
                arm.rm_movej_follow(smoothed)
            else:
                print("code:", ret_ik)

        cv2.imshow("Hand→Arm Joint PBVS", display)
        if cv2.waitKey(1) == ord('q'):
            break

finally:
    pipeline.stop()
    hands.close()
    arm.rm_delete_robot_arm()
    cv2.destroyAllWindows()   

current c api version:  1.1.0
请将手腕移到“零位”并保持不动, 2秒后开始标定…


W0000 00:00:1753611986.213132   21145 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753611986.225476   21145 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


标定零位 Pc0 = [-0.02847892 -0.06855912  0.708     ]


In [ ]:
'''用mediapipe控制夹抓运动,当食指尖与拇指尖的距离小于35mm时闭合反之张开'''
import time
import cv2
import numpy as np
import mediapipe as mp
import pyrealsense2 as rs
from Robotic_Arm.rm_robot_interface import *

# === 参数配置 ===
#D_MAX = 0.07           # 手指最大张开距离 (m)，可通过标定调整
#ALPHA = 0.5            # 低通滤波系数
#SLEEP_TIME = 0.00001       # 循环间隔 (s)

# === 初始化机械臂 & 夹爪行程 ===
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
handle = arm.rm_create_robot_arm("169.254.128.19", 8080)

# === 初始化 RealSense 管道 ===
pipeline = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)
cfg.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
profile = pipeline.start(cfg)

# 对齐深度到彩色
align = rs.align(rs.stream.color)
depth_sensor = profile.get_device().first_depth_sensor()
depth_scale = depth_sensor.get_depth_scale()  # 米/单位

# 获取相机内参
color_stream = profile.get_stream(rs.stream.color).as_video_stream_profile()
intr = color_stream.get_intrinsics()
fx, fy, cx, cy = intr.fx, intr.fy, intr.ppx, intr.ppy

# === 初始化 MediaPipe Hands ===
mp_hands = mp.solutions.hands
mp_draw  = mp.solutions.drawing_utils
hands = mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.7
)

d_filt = None  # 滤波后距离

try:
    while True:
        # 1. 获取对齐帧
        frames = pipeline.wait_for_frames()
        aligned = align.process(frames)
        color_frame = aligned.get_color_frame()
        depth_frame = aligned.get_depth_frame()
        if not color_frame or not depth_frame:
            continue

        color = np.asanyarray(color_frame.get_data())
        h, w, _ = color.shape

        # 2. 手部检测
        img_rgb = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
        res = hands.process(img_rgb)
        if not res.multi_hand_landmarks:
            cv2.imshow("Gripper Control", color)
            if cv2.waitKey(1) == ord('q'): break
            continue

        lm = res.multi_hand_landmarks[0]
        # 拇指指尖与食指指尖
        th = lm.landmark[mp_hands.HandLandmark.THUMB_TIP]
        idx= lm.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
        u_t, v_t = int(th.x*w), int(th.y*h)
        u_i, v_i = int(idx.x*w), int(idx.y*h)
        # 把坐标 clamp 到 [0, w-1], [0, h-1]
        u_t = np.clip(u_t, 0, w-1)
        v_t = np.clip(v_t, 0, h-1)
        u_i = np.clip(u_i, 0, w-1)
        v_i = np.clip(v_i, 0, h-1)
        # 3. 投影到三维
        z_t = depth_frame.get_distance(u_t, v_t)
        z_i = depth_frame.get_distance(u_i, v_i)
        x_t = (u_t - cx) * z_t / fx
        y_t = (v_t - cy) * z_t / fy
        x_i = (u_i - cx) * z_i / fx
        y_i = (v_i - cy) * z_i / fy


        # 4. 计算距离 & 滤波
        d = np.linalg.norm([x_t-x_i, y_t-y_i, z_t-z_i])

        # 当 MediaPipe 检测距离小于 3.5cm 时，强制当作闭合0,反之则张开
        if d < 0.035: 
            arm.rm_set_gripper_position(0, block=True, timeout=0)
        else:
            #arm.rm_set_gripper_release(1000, block=True, timeout=0)
            arm.rm_set_gripper_position(1000, block=True, timeout=0)

        # # clamp 原始距离到最大 70mm
        # d = min(d, D_MAX)
        # d_filt = ALPHA*d + (1-ALPHA)*(d_filt if d_filt is not None else d)
        # # 再 clamp 一次，防止滤波小误差
        # d_filt = min(d_filt, D_MAX)
        # # 5. 线性映射 & 夹爪指令
        # ratio = np.clip(d_filt / D_MAX, 0.0, 1.0)
        # pos = int(ratio * 1000)
        
        # 6. 可视化
        mp_draw.draw_landmarks(color, lm, mp_hands.HAND_CONNECTIONS)
        #cv2.circle(color, (u_t, v_t), 5, (0,255,0), -1)
        #cv2.circle(color, (u_i, v_i), 5, (0,255,0), -1)
        #cv2.putText(color, f"d={d_filt*100:.1f}cm pos={pos}",(10,30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)

        cv2.imshow("Gripper Control", color)
        if cv2.waitKey(1) == ord('q'):
            break

        #time.sleep(SLEEP_TIME)
        

finally:
    # 清理
    hands.close()
    pipeline.stop()
    cv2.destroyAllWindows()
    arm.rm_delete_robot_arm()

current c api version:  1.1.0


W0000 00:00:1753779126.895992   44449 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1753779126.909756   44449 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout error
[rm_set_gripper_position] timeout er